# Load the data

In [17]:
from result_saver import SaverProvider
provider = SaverProvider()

In [18]:
from Scratch import metadata_loader

DEVICE = "ibm_sherbrooke"
DISTANCE = int(40)
ROUNDS = str(30) 
LOGICAL = str(0)

md = metadata_loader(True, True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md.drop(columns=["sampled_state", "num_qubits", "code",  "job_status", "optimization_level", "extra", "tags_xp", "tags"])
md = md[md["backend_name"] == DEVICE]

md = md.dropna(subset=["rounds"])

md = md[md["distance"]==DISTANCE]
# md = md[md["rounds"]==ROUNDS]
md = md[md["logical"]==LOGICAL]

# md

In [32]:
unique_rounds = md['rounds'].unique().tolist()
unique_rounds = [int(r) for r in unique_rounds]
unique_rounds.sort(reverse=True)
print(unique_rounds)

[35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 17, 15, 13, 11, 9, 8, 7, 5]


In [33]:
from tqdm import tqdm
import numpy as np

memories_by_rounds = {rounds: [] for rounds in unique_rounds}
for index, row in tqdm(md.iterrows(), total=md.shape[0], desc="Retrieving memory data"):
    job_id = row['job_id']
    rounds = int(row['rounds'])
    
    # Retrieve the job's memory data. Assuming provider.retrieve_job() is a function that does this.
    memory_data = provider.retrieve_job(job_id).result().get_memory()
    
    # Append the memory data to the corresponding list in the dictionary
    memories_by_rounds[rounds].append(memory_data)

for rounds in tqdm(memories_by_rounds, desc="Concatenating memory data"):
    if len(memories_by_rounds[rounds]) > 1:
        # Concatenate arrays along the first axis
        memories_by_rounds[rounds] = np.concatenate(memories_by_rounds[rounds], axis=0)
    elif len(memories_by_rounds[rounds]) == 1:
        # Just take the single array out of the list
        memories_by_rounds[rounds] = memories_by_rounds[rounds][0]

print(memories_by_rounds.keys())

shot_dict = {rounds: memories_by_rounds[rounds].shape[0] for rounds in memories_by_rounds}
print(shot_dict)

Concatenating memory data: 100%|██████████| 25/25 [00:00<00:00, 709.14it/s]

dict_keys([35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 17, 15, 13, 11, 9, 8, 7, 5])
{35: 10621, 34: 1875, 33: 625, 32: 625, 31: 625, 30: 625, 29: 625, 28: 625, 27: 1250, 26: 625, 25: 1250, 24: 625, 23: 625, 22: 625, 21: 625, 20: 625, 19: 625, 17: 625, 15: 625, 13: 625, 11: 625, 9: 1250, 8: 625, 7: 4196, 5: 625}


# Initialize the code

In [34]:
import stim
import pymatching

from soft_info import get_repcode_layout, get_repcode_IQ_map
from Scratch import create_or_load_kde_grid

layout = get_repcode_layout(distance=DISTANCE, backend=provider.get_backend(DEVICE), _is_hex=True)
grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, 'cn6hk3mxhnxg008djq30', 300, 2, other_date=None) # rndm first sherbrooke job

# takes ~6s

# Decode each round

In [35]:
import cpp_soft_info

p_data = 6.869e-3  # mean sherbrooke noise

# Initialize your dictionary to store results
results_dict_weighted = {rounds: [0, 0] for rounds in unique_rounds}
results_dict_flat_one = {rounds: [0, 0] for rounds in unique_rounds}
results_dict_flat_informed = {rounds: [0, 0] for rounds in unique_rounds}

for synd_rounds in tqdm(memories_by_rounds.keys()):
    circuit = stim.Circuit.generated("repetition_code:memory",
                                     distance=DISTANCE,
                                     rounds=synd_rounds,
                                     after_clifford_depolarization=0.1)
    model = circuit.detector_error_model(decompose_errors=True)
    matching = pymatching.Matching.from_detector_error_model(model)
    qubit_mapping = get_repcode_IQ_map(layout, synd_rounds)

    memory = memories_by_rounds[synd_rounds]

    num_errors_weighted  = cpp_soft_info.decode_IQ_shots(matching._matching_graph, memory, 
                                               synd_rounds, qubit_mapping, grid_dict, 
                                               processed_scaler_dict, p_data=p_data, 
                                               p_mixed=p_data/10,  
                                               common_measure=-1)
    
    num_errors_flat = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, memory, 
                                                        synd_rounds, qubit_mapping, grid_dict, 
                                                        processed_scaler_dict)
    
    num_errors_informed = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, memory, 
                                           synd_rounds, qubit_mapping, grid_dict, processed_scaler_dict,
                                           p_data, p_data/1, p_data/10, common_measure=-1) 
                    
    
    # Store the results in the dictionary
    results_dict_weighted[synd_rounds][0] += num_errors_weighted
    results_dict_weighted[synd_rounds][1] += len(memory)
    results_dict_flat_one[synd_rounds][0] += num_errors_flat
    results_dict_flat_one[synd_rounds][1] += len(memory)
    results_dict_flat_informed[synd_rounds][0] += num_errors_informed
    results_dict_flat_informed[synd_rounds][1] += len(memory)

100%|██████████| 25/25 [04:21<00:00, 10.46s/it]


In [36]:
print(results_dict_weighted)
print(results_dict_flat_one)
print(results_dict_flat_informed)


{35: [30, 10621], 34: [0, 1875], 33: [1, 625], 32: [1, 625], 31: [0, 625], 30: [0, 625], 29: [0, 625], 28: [0, 625], 27: [4, 1250], 26: [1, 625], 25: [1, 1250], 24: [0, 625], 23: [0, 625], 22: [0, 625], 21: [0, 625], 20: [0, 625], 19: [0, 625], 17: [0, 625], 15: [0, 625], 13: [0, 625], 11: [0, 625], 9: [0, 1250], 8: [0, 625], 7: [0, 4196], 5: [0, 625]}
{35: [358, 10621], 34: [16, 1875], 33: [18, 625], 32: [4, 625], 31: [7, 625], 30: [6, 625], 29: [3, 625], 28: [2, 625], 27: [19, 1250], 26: [2, 625], 25: [13, 1250], 24: [0, 625], 23: [1, 625], 22: [2, 625], 21: [0, 625], 20: [1, 625], 19: [2, 625], 17: [0, 625], 15: [0, 625], 13: [0, 625], 11: [1, 625], 9: [0, 1250], 8: [0, 625], 7: [0, 4196], 5: [0, 625]}
{35: [727, 10621], 34: [66, 1875], 33: [48, 625], 32: [19, 625], 31: [16, 625], 30: [19, 625], 29: [12, 625], 28: [10, 625], 27: [53, 1250], 26: [7, 625], 25: [39, 1250], 24: [7, 625], 23: [10, 625], 22: [6, 625], 21: [5, 625], 20: [4, 625], 19: [9, 625], 17: [3, 625], 15: [5, 625], 1